<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

#### ResNet18 모델 정의 및 인스턴스 초기화

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1: # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.leaky_relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.leaky_relu(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.leaky_relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### 데이터셋(Dataset) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


#### 환경 설정 및 학습(Training) 함수 정의

In [5]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr = 0.01
    if epoch >= 75:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습(Training) 진행

* 대략 20번의 epoch 이후에도 85%가량의 test accuracy를 얻을 수 있습니다.

In [6]:
# for epoch in range(0, 200):
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.1171875
Current benign train loss: 2.3607289791107178

Current batch: 100
Current benign train accuracy: 0.265625
Current benign train loss: 1.981872797012329

Current batch: 200
Current benign train accuracy: 0.40625
Current benign train loss: 1.6575568914413452

Current batch: 300
Current benign train accuracy: 0.3125
Current benign train loss: 1.7564576864242554

Total benign train accuarcy: 31.748
Total benign train loss: 733.3147387504578

[ Test epoch: 0 ]

Test accuarcy: 42.95
Test average loss: 0.015371251463890076
Model Saved!

[ Train epoch: 1 ]

Current batch: 0
Current benign train accuracy: 0.40625
Current benign train loss: 1.622209906578064

Current batch: 100
Current benign train accuracy: 0.46875
Current benign train loss: 1.4265668392181396

Current batch: 200
Current benign train accuracy: 0.484375
Current benign train loss: 1.4719771146774292

Current batch: 300
Current benign train accuracy: 0.5


Current batch: 200
Current benign train accuracy: 0.921875
Current benign train loss: 0.2845771014690399

Current batch: 300
Current benign train accuracy: 0.890625
Current benign train loss: 0.2701648473739624

Total benign train accuarcy: 87.716
Total benign train loss: 140.12592814862728

[ Test epoch: 13 ]

Test accuarcy: 83.47
Test average loss: 0.004965446650981903
Model Saved!

[ Train epoch: 14 ]

Current batch: 0
Current benign train accuracy: 0.875
Current benign train loss: 0.33905935287475586

Current batch: 100
Current benign train accuracy: 0.9140625
Current benign train loss: 0.22788187861442566

Current batch: 200
Current benign train accuracy: 0.8359375
Current benign train loss: 0.48903438448905945

Current batch: 300
Current benign train accuracy: 0.8515625
Current benign train loss: 0.4269927442073822

Total benign train accuarcy: 88.27
Total benign train loss: 133.16593879461288

[ Test epoch: 14 ]

Test accuarcy: 85.53
Test average loss: 0.004495028732717037
Mode


Total benign train accuarcy: 91.696
Total benign train loss: 94.86805629730225

[ Test epoch: 26 ]

Test accuarcy: 87.53
Test average loss: 0.003757657944411039
Model Saved!

[ Train epoch: 27 ]

Current batch: 0
Current benign train accuracy: 0.9375
Current benign train loss: 0.20896852016448975

Current batch: 100
Current benign train accuracy: 0.8671875
Current benign train loss: 0.37621694803237915

Current batch: 200
Current benign train accuracy: 0.90625
Current benign train loss: 0.31960242986679077

Current batch: 300
Current benign train accuracy: 0.8984375
Current benign train loss: 0.27682676911354065

Total benign train accuarcy: 91.832
Total benign train loss: 90.95201978832483

[ Test epoch: 27 ]

Test accuarcy: 85.7
Test average loss: 0.004650986644625664
Model Saved!

[ Train epoch: 28 ]

Current batch: 0
Current benign train accuracy: 0.921875
Current benign train loss: 0.1740143895149231

Current batch: 100
Current benign train accuracy: 0.96875
Current benign train 


Current batch: 0
Current benign train accuracy: 0.9375
Current benign train loss: 0.18490582704544067

Current batch: 100
Current benign train accuracy: 0.8515625
Current benign train loss: 0.3713987171649933

Current batch: 200
Current benign train accuracy: 0.9765625
Current benign train loss: 0.08273769915103912

Current batch: 300
Current benign train accuracy: 0.9296875
Current benign train loss: 0.19307030737400055

Total benign train accuarcy: 92.756
Total benign train loss: 81.10387698560953

[ Test epoch: 40 ]

Test accuarcy: 84.87
Test average loss: 0.005126316983997822
Model Saved!

[ Train epoch: 41 ]

Current batch: 0
Current benign train accuracy: 0.9453125
Current benign train loss: 0.11762093752622604

Current batch: 100
Current benign train accuracy: 0.96875
Current benign train loss: 0.13355784118175507

Current batch: 200
Current benign train accuracy: 0.9453125
Current benign train loss: 0.16789817810058594

Current batch: 300
Current benign train accuracy: 0.90625


Current batch: 200
Current benign train accuracy: 0.9921875
Current benign train loss: 0.042661603540182114

Current batch: 300
Current benign train accuracy: 0.984375
Current benign train loss: 0.028816789388656616

Total benign train accuarcy: 98.708
Total benign train loss: 16.079351253807545

[ Test epoch: 53 ]

Test accuarcy: 93.65
Test average loss: 0.0021211147390305996
Model Saved!

[ Train epoch: 54 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.008850958198308945

Current batch: 100
Current benign train accuracy: 0.9921875
Current benign train loss: 0.031599074602127075

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.007446436211466789

Current batch: 300
Current benign train accuracy: 0.9609375
Current benign train loss: 0.071578249335289

Total benign train accuarcy: 98.92
Total benign train loss: 13.203204513993114

[ Test epoch: 54 ]

Test accuarcy: 93.96
Test average loss: 0.002163431563228369
Mode


Total benign train accuarcy: 99.738
Total benign train loss: 4.064403071650304

[ Test epoch: 66 ]

Test accuarcy: 93.93
Test average loss: 0.002397711097076535
Model Saved!

[ Train epoch: 67 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.003921284805983305

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.00812477432191372

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.003273953218013048

Current batch: 300
Current benign train accuracy: 0.984375
Current benign train loss: 0.03490862995386124

Total benign train accuarcy: 99.73
Total benign train loss: 3.751526004925836

[ Test epoch: 67 ]

Test accuarcy: 94.13
Test average loss: 0.0023503483302891254
Model Saved!

[ Train epoch: 68 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.009524877183139324

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0015637632


Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0017506678123027086

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.0009840859565883875

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0021622860804200172

Total benign train accuarcy: 99.922
Total benign train loss: 1.8144584906112868

[ Test epoch: 80 ]

Test accuarcy: 94.44
Test average loss: 0.002419334490969777
Model Saved!

[ Train epoch: 81 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0042364392429590225

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.006954846438020468

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.001573245506733656

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.002860059030354023

Total benign train accuarcy: 99.932
Total benign train loss: 1.6242578068049625


Total benign train accuarcy: 99.964
Total benign train loss: 1.1888875736331102

[ Test epoch: 93 ]

Test accuarcy: 94.42
Test average loss: 0.002450160166248679
Model Saved!

[ Train epoch: 94 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0009061076561920345

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.002584203379228711

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.00311854830943048

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0025344453752040863

Total benign train accuarcy: 99.948
Total benign train loss: 1.3518002604541834

[ Test epoch: 94 ]

Test accuarcy: 94.28
Test average loss: 0.0024759861279278995
Model Saved!

[ Train epoch: 95 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.002067074179649353

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.003338943